<a href="https://colab.research.google.com/github/dhuvik/projects/blob/master/tulip.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
local = False

In [2]:
#@title Clone github repo
import json, time, os, sys, glob

if not local:
  if not os.path.isdir("TULIP-TCR"):
    os.system("git clone -q https://github.com/barthelemymp/TULIP-TCR.git")
  sys.path.append('/content/TULIP-TCR/')

if not local:
  !pip install transformers==4.32.1 datasets  tokenizers

In [3]:
import numpy as np
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers import pipeline
import pandas as pd
import sys

import json
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data._utils.collate import default_collate
#from data import TranslationDataset
from transformers import BertTokenizerFast, BertTokenizer
from transformers import BertModel, BertForMaskedLM, BertConfig, EncoderDecoderModel, BertLMHeadModel, AutoModelForSequenceClassification
from sklearn.metrics import roc_auc_score

import sys
import torch
import torch.utils.data as data
from torch.nn.utils.rnn import pad_sequence
import os


from transformers.models.bert.modeling_bert import BertPreTrainedModel, BertOnlyMLMHead, SequenceClassifierOutput
from torch.nn import MSELoss, CrossEntropyLoss, BCEWithLogitsLoss
from typing import List, Optional, Tuple, Union
from transformers.modeling_outputs import ModelOutput

from transformers import PretrainedConfig
from transformers.modeling_outputs import BaseModelOutput, Seq2SeqLMOutput
from transformers.modeling_utils import PreTrainedModel
from transformers.utils import add_start_docstrings, add_start_docstrings_to_model_forward, logging, replace_return_docstrings
from transformers.models.encoder_decoder.configuration_encoder_decoder import EncoderDecoderConfig
import warnings

from src.multiTrans import TulipPetal, TCRDataset, BertLastPooler, unsupervised_auc, train_unsupervised, eval_unsupervised, MyMasking, Tulip, get_logscore


/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


# Load model and file

In [4]:
from google.colab import files
# uploaded = files.upload()
# filesname = list(uploaded.keys())[0]
finetunedmodel = True #@param {type:"boolean"}
with open("/content/TULIP-TCR/configs/shallow.config.json", "r") as read_file:
    print("loading hyperparameter")
    modelconfig = json.load(read_file)


loading hyperparameter


# run prediction

In [6]:
torch.manual_seed(0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Using device:", device)

checkpoint = torch.load("/content/TULIP-TCR/model_weights/pytorch_model.bin", map_location='cpu')

tokenizer = AutoTokenizer.from_pretrained("/content/TULIP-TCR/aatok/")
if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '<PAD>'})

if tokenizer.sep_token is None:
    tokenizer.add_special_tokens({'sep_token': '<MIS>'})

if tokenizer.cls_token is None:
    tokenizer.add_special_tokens({'cls_token': '<CLS>'})

if tokenizer.eos_token is None:
    tokenizer.add_special_tokens({'eos_token': '<EOS>'})

if tokenizer.mask_token is None:
    tokenizer.add_special_tokens({'mask_token': '<MASK>'})



from tokenizers.processors import TemplateProcessing
tokenizer._tokenizer.post_processor = TemplateProcessing(
    single="<CLS> $A <EOS>",
    pair="<CLS> $A <MIS> $B:1 <EOS>:1",
    special_tokens=[
        ("<EOS>", 2),
        ("<CLS>", 3),
        ("<MIS>", 4),
    ],
)

mhctok = AutoTokenizer.from_pretrained("/content/TULIP-TCR/mhctok/")



# mhctok = AutoTokenizer.from_pretrained("mhctok/")

vocabsize = len(tokenizer._tokenizer.get_vocab())
mhcvocabsize = len(mhctok._tokenizer.get_vocab())
print(mhcvocabsize)
print("Loading models ..")
# vocabsize = encparams["vocab_size"]

max_length = 50
encoder_config = BertConfig(vocab_size = vocabsize,
                    max_position_embeddings = max_length, # this shuold be some large value
                    num_attention_heads = modelconfig["num_attn_heads"],
                    num_hidden_layers = modelconfig["num_hidden_layers"],
                    hidden_size = modelconfig["hidden_size"],
                    type_vocab_size = 1,
                    pad_token_id =  tokenizer.pad_token_id)

encoder_config.mhc_vocab_size  =mhcvocabsize

encoderA = BertModel(config=encoder_config)
encoderB = BertModel(config=encoder_config)
encoderE = BertModel(config=encoder_config)

max_length = 100
max_length = 50
decoder_config = BertConfig(vocab_size = vocabsize,
                    max_position_embeddings = max_length, # this shuold be some large value
                    num_attention_heads = modelconfig["num_attn_heads"],
                    num_hidden_layers = modelconfig["num_hidden_layers"],
                    hidden_size = modelconfig["hidden_size"],
                    type_vocab_size = 1,
                    is_decoder=True,
                    pad_token_id =  tokenizer.pad_token_id)    # Very Important

decoder_config.add_cross_attention=True

decoderA = TulipPetal(config=decoder_config) #BertForMaskedLM
decoderA.pooler = BertLastPooler(config=decoder_config)
decoderB = TulipPetal(config=decoder_config) #BertForMaskedLM
decoderB.pooler = BertLastPooler(config=decoder_config)
decoderE = TulipPetal(config=decoder_config) #BertForMaskedLM
decoderE.pooler = BertLastPooler(config=decoder_config)


# Define encoder decoder model
model = Tulip(encoderA=encoderA,encoderB=encoderB,encoderE=encoderE, decoderA=decoderA, decoderB=decoderB, decoderE=decoderE)

model.load_state_dict(checkpoint)
model.to(device)

Using pad_token, but it is not set yet.
Using sep_token, but it is not set yet.
Using cls_token, but it is not set yet.
Using eos_token, but it is not set yet.
Using mask_token, but it is not set yet.


Using device: cpu
55
Loading models ..
self.pad_token_id 1
self.pad_token_id 1
self.pad_token_id 1


Tulip(
  (encoderA): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30, 128, padding_idx=1)
      (position_embeddings): Embedding(50, 128)
      (token_type_embeddings): Embedding(1, 128)
      (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-1): 2 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=128, out_features=128, bias=True)
              (key): Linear(in_features=128, out_features=128, bias=True)
              (value): Linear(in_features=128, out_features=128, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=128, out_features=128, bias=True)
              (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)


In [7]:
import transformers

print(transformers.__version__)

4.32.1


In [8]:
vdj_filesname = "/content/TULIP-TCR/data/VDJ_test_2.csv"
compute_auc=True
from sklearn.metrics import roc_auc_score
pd.read_csv(vdj_filesname)

,Unnamed: 0,CDR3b,CDR3a,peptide,MHC,binder
0,0,CASSIRASYEQYF,CAGASGNTGKLIF,GILGFVFTL,HLA-A*02:01,1
1,1,CASSIRSTDTQYF,CAGSGGGSQGNLIF,GILGFVFTL,HLA-A*02:01,1
2,2,CASSIRAADTQYF,CALDMGGGSQGNLIF,GILGFVFTL,HLA-A*02:01,1
3,3,CATSSGTGAYEQYF,CVVNAHDMRF,LLWNGPMAV,HLA-A*02:01,1
4,4,CASSDASGSIGEQFF,CAVTVSNDYKLSF,CINGVCWTV,HLA-A*02:01,1
...,...,...,...,...,...,...
6127,6127,CASSWDRVLYEQYF,CSNYKLTF,GILGFVFTL,HLA-A*02:01,0
6128,6128,CASSDRLENNEKLFF,CAVTDNNNNDMRF,GILGFVFTL,HLA-A*02:01,0
6129,6129,CASSQGQEKETQYF,CAVENYQLIW,GILGFVFTL,HLA-A*02:01,0
6130,6130,CSARDQQGQNTGELFF,CVVNRGDGLTF,GILGFVFTL,HLA-A*02:01,0


In [9]:
!wget 'https://github.com/dhuvik/projects/raw/master/projects/IMMREP23_solutions.csv'
!wget https://github.com/barthelemymp/TULIP-TCR/raw/main/data/SeenPeptides.zip

--2025-05-22 03:38:28--  https://github.com/dhuvik/projects/raw/master/projects/IMMREP23_solutions.csv
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dhuvik/projects/master/projects/IMMREP23_solutions.csv [following]
--2025-05-22 03:38:28--  https://raw.githubusercontent.com/dhuvik/projects/master/projects/IMMREP23_solutions.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2601402 (2.5M) [text/plain]
Saving to: ‘IMMREP23_solutions.csv.1’

IMMREP23_solutions. 100%[===================>]   2.48M  --.-KB/s    in 0.05s   

2025-05-22 03:38:28 (50.9 MB/s) - ‘IMMREP23_solutions.csv.1’ saved [260140

In [10]:
!unzip SeenPeptides.zip

Archive:  SeenPeptides.zip
replace SeenPeptides/tcr2b.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/SeenPeptides/._tcr2b.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace SeenPeptides/tulip2_finetune02_bichain.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace SeenPeptides/tcr3b.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/SeenPeptides/._tcr3b.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace SeenPeptides/tcr4b.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/SeenPeptides/._tcr4b.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace SeenPeptides/tcr5b.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/SeenPeptides/._tcr5b.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace SeenPeptides/tulip2_pretune.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace SeenPeptides/scripts.rtf? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace __MACOSX/SeenPeptides/._scripts.rtf? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace SeenPeptides/

In [13]:
!ls SeenPeptides/

NetMHCpan_1.csv    tcr1b.csv  tcr6b.csv
NetMHCpan_2.csv    tcr2b.csv  tulip2_finetune02_bichain.csv
NetMHCpan_3.csv    tcr3b.csv  tulip2_pretune.csv
NetMHCpan_456.csv  tcr4b.csv
scripts.rtf	   tcr5b.csv


In [19]:
seen_epitopes = pd.read_csv('./SeenPeptides/tulip2_finetune02_bichain.csv')['peptide'].unique()
seen_epitopes

array(['TPAAPAPA', 'CINGVCWTV', 'LLWNGPMAV', 'NLVPMVATV', 'GILGFVFTL',
       'GLCTLVAML', 'VLFGLGFAI', 'LLFGYPVYV', 'AVYDGREHTV', 'IMNDMPIYM',
       'FLCMKALLL', 'FLYALALLL', 'YVLDHLIVV', 'SLLMWITQV', 'SLLMWITQC',
       'ELAGIGILTV', 'AAGIGILTV', 'ALWGPDPAAA', 'ALGIGILTV', 'ILAKFLHWL',
       'EAAGIGILTV', 'ALWGFFPVL', 'KLVALVINAV', 'YQFGPDFPIA',
       'RQWGPDPAAV', 'YLGGPDFPTI', 'MVWGPDPLYV', 'RQFGPDFPTI',
       'RQFGPDWIVA', 'YLEPGPVTV', 'AQWGPDPAAA', 'KLVALGINAV', 'SLYNTVATL',
       'SLYNTIATL', 'SLFNTIAVL', 'SLSKILDTV', 'GVYDGREHTV', 'ITDQVPFSV',
       'IMDQVPFSV', 'ILDQVPFSV', 'HMTEVVRHC', 'RLQSLQTYV', 'RLITGRLQSL',
       'TLATHGLAAV', 'MQLFFSYFA', 'ALQIPFAMQM', 'KIYSKHTPI', 'MLAKALRKV',
       'MLYQHLLPL', 'RLARLALVL', 'AVGSYVYSV', 'NLNCCSVPV', 'ALYGFVPVL',
       'VVMSWAPPV', 'KLFEFLVYGV', 'CVNGSCFTV', 'RLGPVQNEV', 'KLVAMGINAV',
       'YLSNIIPAL', 'QLCDVMFYL', 'YLYDRLLRV', 'KLYGLDWAEL', 'ALDPHSGHFV',
       'ALSPVIPHI', 'LLDFVRFMGV', 'FLASKIGRLV', 'MLDLQPETT', 'YLLEMLWR

In [20]:
immrep_filesname = 'IMMREP23_solutions.csv'
immrep23 = pd.read_csv(immrep_filesname)
immrep23 = immrep23.rename(columns={'Label':'binder', 'Peptide':'peptide', 'HLA':'MHC'})
immrep23['MHC'] = ['HLA-' + x for x in immrep23['MHC']]
immrep23=immrep23[~immrep23['peptide'].isin(seen_epitopes)]
immrep23.to_csv('IMMREP23_solutions.csv', index=False)

In [30]:
def mask_features(df, feats):
    new_df = df.copy()
    for feat in feats:
        new_df[feat] = '<MIS>'
    return new_df


mask_features(immrep23, ['CDR3a', 'MHC']).to_csv('IMMREP23_CDR3b_only.csv', index=False)
mask_features(immrep23, ['MHC']).to_csv('IMMREP23_CDR3b_CDR3a.csv', index=False)
immrep23[['CDR3b', 'CDR3a', 'peptide', 'MHC', 'binder']].to_csv('IMMREP23_CDR3ab_mhc.csv', index=False)

In [54]:
!pip install levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.5 MB/s eta 0:00:00


In [56]:
import Levenshtein as lev


def find_closest_sequences(query_seqs, ref_seqs):
    results = []
    for query in query_seqs:
        # Initialize minimum distance with a very high value
        min_distance = float('inf')
        closest_seqs = []
        # Compare each query with all reference sequences
        for ref in ref_seqs:
            distance = lev.distance(query, ref)
            if distance < min_distance:
                # New minimum found, reset the list of closest sequences
                min_distance = distance
                closest_seqs = [ref]
            elif distance == min_distance:
                # Tie found, add this sequence to the list
                closest_seqs.append(ref)
        # Store the result for this query
        results.append((query, closest_seqs, min_distance))
    return results

In [59]:
def run_tulip_predictions(filesname):
    target_peptidesFinal = pd.read_csv(filesname)["peptide"].unique()
    closest_peptides = find_closest_sequences(target_peptidesFinal, seen_epitopes)
    auces = []
    for i, target_peptide in enumerate(target_peptidesFinal):
        results = pd.DataFrame(columns=["CDR3a", "CDR3b", "peptide", "rank"])
        datasetPetideSpecific= TCRDataset(filesname, tokenizer, device,target_peptide=target_peptide, mhctok=mhctok)
        find_closest_sequences
        print(closest_peptides[i])
        scores = -1*np.array(get_logscore(datasetPetideSpecific, model, ignore_index =  tokenizer.pad_token_id))
        ranks = np.argsort(np.argsort(scores))
        results["CDR3a"] = datasetPetideSpecific.alpha
        results["CDR3b"] = datasetPetideSpecific.beta
        results["Peptide"] = target_peptide
        results["rank"] = ranks
        # print(results)
        if compute_auc:
            dl = torch.utils.data.DataLoader(dataset=datasetPetideSpecific, batch_size=1, shuffle=False, collate_fn=datasetPetideSpecific.all2allmhc_collate_function)
            print(unsupervised_auc(model,dl, tokenizer.pad_token_id))
            auce = roc_auc_score(datasetPetideSpecific.binder, ranks)
            auces += [(closest_peptides[i][0], closest_peptides[i][2], auce)]
            print(f'AUC: {auce}')
        else:
            print(results)
        #results.to_csv('yourpath')
    return auces

In [60]:
auces = run_tulip_predictions('IMMREP23_CDR3b_CDR3a.csv')

Loading the data ...
('SALPTNADLY', ['SLYNTVATLY', 'ALPETTADI'], 5)
(np.float64(0.4253086419753086), np.float64(0.441358024691358), np.float64(0.5191358024691358))
AUC: 0.5191358024691358
Loading the data ...
('IPSINVHHY', ['ILKEPVHGV', 'ALSKGVHFV', 'MLNIPSINV'], 6)
(np.float64(0.47578125000000004), np.float64(0.46171875), np.float64(0.44140625))
AUC: 0.44140625
Loading the data ...
('RAKFKQLL', ['RLSFKELLV'], 4)
(np.float64(0.44252688172043014), np.float64(0.5374193548387096), np.float64(0.5980107526881719))
AUC: 0.5980107526881719
Loading the data ...
('EPLPQGQLTAY', ['ALLPGLPAA'], 6)
(np.float64(0.41050347222222217), np.float64(0.6143229166666667), np.float64(0.5220486111111111))
AUC: 0.5220486111111111
Loading the data ...
('VLEETSVML', ['MLDMYSVML', 'VLASPSVPL'], 4)
(np.float64(0.40816326530612246), np.float64(0.44988662131519275), np.float64(0.5755102040816327))
AUC: 0.5755102040816327
Loading the data ...
('FTDALGIDEY', ['KLVALGINAV', 'KLSALGINAV', 'FLALCADSI', 'YTVELGTEV', 'SLV

In [61]:
np.mean(auces)

np.float64(0.5434634309584194)

In [ ]:
x = []
y = []